## Librerías

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import lightgbm as lgb




from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree,  _tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.metrics import f1_score

from umap import UMAP
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.ensemble import  RandomForestClassifier
from sklearn.impute import SimpleImputer
from joblib import Parallel, delayed
from time import time
import duckdb
# %load_ext sql
# %config SqlMagic.autopandas = True
# %config SqlMagic.feedback = False
# %config SqlMagic.displaycon = False
# %sql duckdb:///:memory:
import time
import optuna
from optuna.study import MaxTrialsCallback
from optuna.trial import TrialState

from optuna.visualization import plot_param_importances, plot_contour,  plot_slice, plot_optimization_history
import plotly.express as px
import os

import kaggle 
import glob



In [2]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [3]:
import random

numeros_random = random.sample(range(1, 1000), 100)


In [4]:
semillas =[ 400009,  500009, 500011, 500021, 600009, 600011, 600021, 700009, 7000011,  700021, 800009, 800021]


In [5]:
# import duckdb


con = duckdb.connect(database=':memory:', read_only=False)

ruta_csv = r'c:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\datasets\competencia_1\competencia_01_crudo.csv'

con.execute(f"CREATE TABLE df_train AS SELECT * FROM read_csv_auto('{ruta_csv}')")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
#dataset perteneciente a la carpeta datasets del repisotorio, archivo: competencia_01.csv
df_train = pd.read_csv(r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\Git\dmeyf2024\datasets\competencia_01_crudo.csv")

In [7]:
df_parallel = pd.read_csv(r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\Git\dmeyf2024\datasets\competencia_01_crudo.csv")

start_time = time.time()

# Crear la columna clase_ternaria usando la columna 'foto_mes' basada en la presencia real

df_ternaria= duckdb.query('''

    WITH siguiente_mes AS (

        SELECT

            numero_de_cliente,

            foto_mes,

            -- Obtener los meses siguientes en los que el cliente está presente

            LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_proximo1,

            LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_proximo2

        FROM df_parallel

    )

    SELECT

        numero_de_cliente,

        foto_mes,

        CASE

            -- Si el cliente está en los dos meses consecutivos siguientes

            WHEN foto_mes_proximo1 = foto_mes + 1 AND foto_mes_proximo2 = foto_mes + 2 THEN 'CONTINUA'

            -- Si el cliente no está en el siguiente mes

            WHEN foto_mes_proximo1 IS NULL OR foto_mes_proximo1 <> foto_mes + 1 THEN 'BAJA+1'

            -- Si el cliente está en el siguiente mes pero no en el segundo mes consecutivo

            WHEN foto_mes_proximo1 = foto_mes + 1 AND (foto_mes_proximo2 IS NULL OR foto_mes_proximo2 <> foto_mes + 2) THEN 'BAJA+2'

            ELSE NULL

        END AS clase_ternaria

    FROM siguiente_mes

    ORDER BY numero_de_cliente, foto_mes

    ''')

df_ternaria = df_ternaria.to_df()
df_ternaria = df_ternaria[['numero_de_cliente', 'foto_mes', 'clase_ternaria']]


df_parallel = df_parallel.merge(df_ternaria, on=['numero_de_cliente', 'foto_mes'], how='left')

df_train = df_parallel



In [8]:
pip install statsmodels

In [9]:
df_train.columns.values

array(['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip',
       'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad',
       'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen',
       'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente',
       'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro',
       'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares',
       'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito',
       'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa',
       'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo',
       'ctarjeta_master', 'ctarjeta_master_transacciones',
       'mtarjeta_master_consumo', 'cprestamos_personales',
       'mprestamos_personales', 'cprestamos_prendarios',
       'mprestamos_prendarios', 'cprestamos_hipotecarios',
       'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares',
       'mplazo_fijo_pesos', 'cinversion1', 'minversi

In [10]:
df_train.head()

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,249221109,202101,1,0,0,61,300,1664.94,17994.40,1605.81,...,4.0,0.0,-37098.90,0.0,7867.0,118514.88,10.0,0.0,76538.25,CONTINUA
1,249221468,202101,1,0,0,53,37,4879.78,18640.95,1149.08,...,4.0,0.0,-9864.93,0.0,1124.0,31427.14,14.0,0.0,0.00,CONTINUA
2,249223005,202101,1,0,0,48,202,1828.35,37789.01,610.12,...,4.0,0.0,-32119.08,0.0,3486.0,855.11,2.0,0.0,19389.69,CONTINUA
3,249228180,202101,1,0,0,66,320,175.87,2600.67,276.22,...,4.0,0.0,-10891.30,0.0,8925.0,7517.73,6.0,0.0,516.12,CONTINUA
4,249232117,202101,1,0,0,79,373,16600.69,32270.07,16367.13,...,4.0,0.0,-43610.75,0.0,8607.0,37309.25,5.0,0.0,3084.99,CONTINUA


In [11]:
df_train.select_dtypes(exclude=['object']).dtypes

numero_de_cliente            int64
foto_mes                     int64
active_quarter               int64
cliente_vip                  int64
internet                     int64
                            ...   
Visa_fechaalta             float64
Visa_mconsumototal         float64
Visa_cconsumos             float64
Visa_cadelantosefectivo    float64
Visa_mpagominimo           float64
Length: 154, dtype: object

df_train.columns.values

# Feature Engineering

In [12]:
lista_feature = ['active_quarter','mrentabilidad','mrentabilidad_annual','mcuenta_corriente','mcaja_ahorro','mcuentas_saldo','ctarjeta_debito_transacciones','mautoservicio','ctarjeta_visa_transacciones','mtarjeta_visa_consumo','mtarjeta_master_consumo','cprestamos_personales','mprestamos_prendarios','cprestamos_hipotecarios','mprestamos_hipotecarios','cplazo_fijo','mplazo_fijo_dolares','mplazo_fijo_pesos','cpayroll_trx','mpayroll','mpayroll2','cpayroll2_trx','ccuenta_debitos_automaticos','mcuenta_debitos_automaticos','ctarjeta_visa_debitos_automaticos','mttarjeta_visa_debitos_automaticos','ctarjeta_master_debitos_automaticos','mttarjeta_master_debitos_automaticos','cpagodeservicios','mpagodeservicios','mpagomiscuentas','cpagomiscuentas','ccajeros_propios_descuentos','mcajeros_propios_descuentos','ctarjeta_visa_descuentos','mtarjeta_visa_descuentos','ctarjeta_master_descuentos','mtarjeta_master_descuentos','ccomisiones_mantenimiento','mcomisiones_mantenimiento','ccomisiones_otras','mcomisiones_otras','cforex','cforex_buy','mforex_buy','cforex_sell','mforex_sell','ctransferencias_recibidas','mtransferencias_recibidas','ctransferencias_emitidas','mtransferencias_emitidas','cextraccion_autoservicio','mextraccion_autoservicio','ccheques_depositados','mcheques_depositados','ccheques_emitidos','mcheques_emitidos','ccheques_depositados_rechazados','mcheques_depositados_rechazados','ccheques_emitidos_rechazados','mcheques_emitidos_rechazados','ccallcenter_transacciones','chomebanking_transacciones','ccajas_transacciones','ccajas_consultas','ccajas_depositos','ccajas_extracciones','ccajas_otras','catm_trx','matm','catm_trx_other','matm_other','ctrx_quarter','Master_Fvencimiento','Master_msaldototal','Master_msaldopesos','Master_msaldodolares','Master_mconsumospesos','Master_mconsumosdolares','Master_mlimitecompra','Master_madelantopesos','Master_madelantodolares','Master_fultimo_cierre','Master_mpagado','Master_mpagospesos','Master_mpagosdolares','Master_fechaalta','Master_mconsumototal','Master_cconsumos','Master_cadelantosefectivo','Master_mpagominimo','Visa_mfinanciacion_limite','Visa_Fvencimiento','Visa_Finiciomora','Visa_msaldototal','Visa_msaldopesos','Visa_msaldodolares','Visa_mconsumospesos','Visa_mconsumosdolares','Visa_mlimitecompra','Visa_madelantopesos', 'Visa_madelantodolares','Visa_fultimo_cierre','Visa_mpagado','Visa_mpagospesos','Visa_mpagosdolares','Visa_fechaalta','Visa_mconsumototal','Visa_cconsumos','Visa_cadelantosefectivo','Visa_mpagominimo']

In [13]:

for x in lista_feature:
    if x not in df_train.columns:
        print(f'{x} está mal escrita')

In [14]:
# con.execute('''
# CREATE OR REPLACE TABLE df_train AS
# SELECT *,
#     (COALESCE(mpayroll, 0) - COALESCE(mcuentas_saldo, 0)) / NULLIF(ctrx_quarter, 0) AS intensidad_gasto
# FROM df_train
# ''')

In [15]:
# df_train['intensidad_gasto'] = (df_train['mpayroll'].fillna(0) - df_train['mcuentas_saldo'].fillna(0)) / df_train['ctrx_quarter'].replace(0, pd.NA)

In [16]:
# con.execute('''
# create or replace table df_train as
# select 
#     *,
#     (IFNULL(visa_msaldototal, 0) + IFNULL(master_msaldototal, 0)) as t_saldo_total
# FROM df_train''')




In [17]:
# df_train['t_saldo_total'] = df_train['Visa_msaldototal'].fillna(0) + df_train['Master_msaldototal'].fillna(0)

In [18]:
# con.execute('''
# create or replace table df_train as
# select 
#     *,
#     (ifnull(Master_mconsumototal,0) + ifnull(Visa_mconsumototal,0)) as t_consumo_total
# from df_train''')


In [19]:
# con.execute('''
#             create or replace table df_train as
#             select 
#                     *,
#             (IFNULL(t_consumo_total,0) - IFNULL(mpayroll,0)) /NULLIF((IFNULL(mcuentas_saldo, 0) - IFNULL(t_saldo_total,0)), 0) as nivel_endeudamiento, 
#             FROM df_train''')

In [20]:
# df_train['t_consumo_total'] = df_train['Master_mconsumototal'].fillna(0) + df_train['Visa_mconsumototal'].fillna(0)

### Status tarjeta

In [21]:
con.execute('''
create or replace table df_train as
select
    *,
    (case when visa_status = 0 then True end) as T_Visa_normal,
    (case when Master_status = 0 then True end) as T_Master_normal
from df_train''')


con.execute(''' create or replace table df_train as
            select *,
            CASE 
            WHEN visa_status = 6 then 1
            WHEN visa_status = 7 then 2 
            WHEN visa_status = 8 then 3
            ELSE NULL
            END as Visa_status,
            from df_train''')

In [22]:
con.execute('''alter table df_train drop column visa_status''')
con.execute('''alter table df_train drop column Master_status''')

In [23]:
# df_train['T_Visa_normal'] = df_train['Visa_status'] == 0


# df_train['T_Master_normal'] = df_train['Master_status'] == 0

In [24]:
# con.execute('''
# create or replace table df_train as
# select *
#     , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
#     , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
# from df_train''')

In [25]:
# con.execute('''create or replace table df_train as
#     select *
#     , Master_mpagado+Visa_mpagado as tarjeta_total_pagado
#     from df_train''')

In [26]:
# con.execute('''create or replace table df_train as
#     select *
#     ,IFNULL(tarjeta_total_pagado, 0) / NULLIF(IFNULL(mprestamos_personales,0),0) as actitud_pago
#             from df_train''')

In [27]:
# con.execute('''
# create or replace table df_train as
# select *,
#     lag (actitud_pago, 1) over (
#         partition by numero_de_cliente 
#         order by foto_mes
#     ) as lag_actitud_pago_1,
#     lag (actitud_pago, 2) over (
#         partition by numero_de_cliente 
#         order by foto_mes
#     ) as lag_actitud_pago_2
# from df_train''')

In [28]:
# df_train['T_Visa_normal'] = df_train['Visa_status'] == 0

# # Calcular la nueva columna T_Master_normal
# df_train['T_Master_normal'] = df_train['Master_status'] == 0

### FE lags

In [29]:
df_train.shape

(981946, 155)

In [30]:
for variable in lista_feature:
    nombre_variable = variable + '_lag_1'
    nombre_variable2 = variable + '_lag_2'
    con.execute(f'''
    create or replace table df_train as
    select *,
        lag ({variable}, 1) over (
            partition by numero_de_cliente 
            order by foto_mes
        ) as {nombre_variable},
        lag ({variable}, 2) over (
            partition by numero_de_cliente 
            order by foto_mes
        ) as {nombre_variable2}
    from df_train''')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [38]:
ruta_guardado= r'C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\Git\dmeyf2024\datasets\competencia_02_f_e_lags.csv'

In [39]:
con.execute(f"""
    COPY (
        SELECT * FROM df_train
    ) TO '{ruta_guardado}'
    (FORMAT 'parquet')
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [37]:
con.execute(f'''EXPORT df_train '{ruta_guardado}' FORMAT PARQUET''')

ParserException: Parser Error: syntax error at or near "df_train"

In [ ]:

# con.execute('''
# create or replace table df_train as
# select *,
#     lag (mpayroll, 1) over (
#         partition by numero_de_cliente 
#         order by foto_mes
#     ) as lag_mpayroll1,
#     lag (mpayroll, 2) over (
#         partition by numero_de_cliente 
#         order by foto_mes
#     ) as lag_mpayroll2
# from df_train''')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [81]:

# df_sorted = df_train.sort_values(by=['numero_de_cliente', 'foto_mes'])


# df_train['lag_ctrx_quarter1'] = df_sorted.groupby('numero_de_cliente')['ctrx_quarter'].shift(1)


# df_train['lag_ctrx_quarter2'] = df_sorted.groupby('numero_de_cliente')['ctrx_quarter'].shift(2)

In [82]:
# con.execute('''
# create or replace table df_train as
# select * 
#     , lag (t_consumo_total, 1) over (
#         partition by numero_de_cliente 
#         order by foto_mes, numero_de_cliente
#     ) as lag_consumo_total1
#     , lag (t_consumo_total, 2) over (
#         partition by numero_de_cliente 
#         order by foto_mes, numero_de_cliente
#     ) as lag_consumo_total2
# from df_train''')

### FE Deltas

In [ ]:
con.execute('''create or replace table df_train as
select *
  , ctrx_quarter - lag_ctrx_quarter1 as delta_1_ctrx_quarter
  , ctrx_quarter - lag_ctrx_quarter2 as delta_1_ctrx_quarter
from df_train''')

In [84]:

# con.execute('''create or replace table df_train as
# select *
#   , t_consumo_total - lag_consumo_total1 as delta_1_lag_confumo_total
#   , t_consumo_total - lag_consumo_total2 as delta_2_lag_consumo_total
# from df_train''')

In [85]:
# con.execute(''' create or replace table df_train as
# select *
#   ,         IFNULL (t_consumo_total,0) / NULLIF (IFNULL (mpayroll,0),0) as consumo_vs_payroll
#             from df_train''')

In [86]:
# con.execute('''
# create or replace table df_train as
# select * 
#     , lag (consumo_vs_payroll, 1) over (
#         partition by numero_de_cliente 
#         order by foto_mes, numero_de_cliente
#     ) as lag_consumo_vs_payroll_1
#     , lag (consumo_vs_payroll, 2) over (
#         partition by numero_de_cliente 
#         order by foto_mes, numero_de_cliente
#     ) as lag_consumo_vs_payroll_2
# from df_train''')

In [87]:
# # Ordenar el DataFrame por 'numero_de_cliente' y 'foto_mes'
# df_sorted = df_train.sort_values(by=['numero_de_cliente', 'foto_mes'])

# # Calcular la columna 'lag_consumo_total1'
# df_train['lag_consumo_total1'] = df_sorted.groupby('numero_de_cliente')['t_consumo_total'].shift(1)

# # Calcular la columna 'lag_consumo_total2'
# df_train['lag_consumo_total2'] = df_sorted.groupby('numero_de_cliente')['t_consumo_total'].shift(2)

### FE Percents

In [33]:
for variable in lista_feature:
    
    nombre_variable= variable + '_percent_rank'    
    con.execute(f'''
    create or replace table df_train as
    select *
        , percent_rank() over (partition by foto_mes order by {variable},numero_de_cliente) as {nombre_variable}
    from df_train''')



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [89]:
# con.execute('''create or replace table df_train as
# select *, 
#             greatest(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as max_tarjeta_transacciones,
#             greatest(ccallcenter_transacciones, chomebanking_transacciones) as max_dig_analog_transacciones,
#             least(ctarjeta_debito_transacciones, ctarjeta_visa_transacciones) as min_tarjeta_debito_transacciones,
#             least(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as min_tarjeta_master_transacciones,
#             least(ccallcenter_transacciones, chomebanking_transacciones) as min_callcenter_transacciones,
#             from df_train''')
            

In [90]:
# # Calcular percent_rank para cada columna
# df_train['cliente_antiguedad_dd'] = df_train.groupby('foto_mes')['cliente_antiguedad'].rank(method='min', pct=True)
# df_train['cliente_edad_dd'] = df_train.groupby('foto_mes')['cliente_edad'].rank(method='min', pct=True)
# df_train['ctrx_quarter_dd'] = df_train.groupby('foto_mes')['ctrx_quarter'].rank(method='min', pct=True)
# df_train['mactivos_margen_dd'] = df_train.groupby('foto_mes')['mactivos_margen'].rank(method='min', pct=True)
# df_train['mpayroll_dd'] = df_train.groupby('foto_mes')['mpayroll'].rank(method='min', pct=True)
# df_train['intensidad_gasto_dd'] = df_train.groupby('foto_mes')['intensidad_gasto'].rank(method='min', pct=True)
# df_train['cproductos_dd'] = df_train.groupby('foto_mes')['cproductos'].rank(method='min', pct=True)

In [91]:
# df_train = con.execute('''SELECT * FROM df_train''').fetch_df()


In [92]:
# con.close()

In [93]:
# df_train.shape

In [94]:
# # df_train.to_csv(r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\Git\dmeyf2024\datasets\competencia_03_f_e.csv", index=False)
# df_train = pd.read_csv(r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\Git\dmeyf2024\datasets\competencia_03_f_e.csv", index_col=False)    

### Modelo para variables más importantes + optimización 

In [96]:
# optuna.visualization.plot_optimization_history(study)

In [97]:
# plot_param_importances(study)

In [98]:
# plot_slice(study)

In [99]:
# best_params = study.best_params
# print("Mejores parámetros:", best_params)

# Light GBM

## Formateo pre modelo

In [100]:

ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = 202104
mes_test = 202106


In [101]:
data = df_train

In [102]:
data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [45]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

In [46]:
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)

In [48]:

df_train = data[data['foto_mes']<=202104]
df_test = data[data['foto_mes']==202106]


In [49]:
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']

In [104]:
w_train = df_train.loc[X_train.index, 'clase_peso']


### Optimización

In [105]:
def objective(trial):
    num_leaves = trial.suggest_int('num_leaves', 8, 100)
    learning_rate = trial.suggest_float('learning_rate', 0.00005, 0.3)  # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 2000)
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0)

    params = {
        'objective': 'binary',
        'metric': 'custom', 
        'boosting_type': 'gbdt',
        'device': 'gpu',  # Usar la GPU
        'gpu_platform_id': 0,  # (opcional) ID de la plataforma GPU
        'gpu_device_id': 0,     # (opcional) ID del dispositivo GPU
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'device': 'gpu',
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }

    train_data = lgb.Dataset(X_train,
                             label=Y_train,  # elegir la clase
                             weight=w_train)
                             
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=300,  # modificar, subir y subir... y descomentar la línea inferior
        
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )

    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5


storage_name = r"sqlite:///C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\Optuna\rf_segmentacion1\optimization_tree.db"
study_name = "competencia_02"



study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

import optuna

class EarlyStoppingByImprovement:
    def __init__(self, patience: int):
        """
        Args:
            patience (int): Número de pruebas consecutivas sin mejora antes de detener la optimización.
        """
        self.patience = patience
        self.best_value = None
        self.no_improvement_trials = 0

    def __call__(self, study, trial):
        current_best_value = study.best_trial.value

        # Si la métrica mejora, reseteamos el contador
        if self.best_value is None or current_best_value > self.best_value:
            self.best_value = current_best_value
            self.no_improvement_trials = 0
        else:
            # Si no hay mejora, incrementamos el contador
            self.no_improvement_trials += 1

        # Detener si no ha habido mejora en 'self.patience' pruebas consecutivas
        if self.no_improvement_trials >= self.patience:
            print(f"Early stopping: No hay mejora en {self.patience} pruebas consecutivas.")
            study.stop()

# Crear el callback con 100 pruebas consecutivas sin mejora
early_stopping_callback = EarlyStoppingByImprovement(patience=100)

# Crear el estudio
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

# Optimizar el estudio con el callback personalizado
study.optimize(objective, n_trials=30000)

[I 2024-11-08 11:24:00,504] Using an existing study with name 'competencia_02' instead of creating a new one.
[I 2024-11-08 11:24:00,548] Using an existing study with name 'competencia_02' instead of creating a new one.
[I 2024-11-08 11:25:26,175] Trial 1 finished with value: 484022000.0 and parameters: {'num_leaves': 73, 'learning_rate': 0.048381168713106194, 'min_data_in_leaf': 1888, 'feature_fraction': 0.9611290073992866, 'bagging_fraction': 0.5702627562564301}. Best is trial 1 with value: 484022000.0.
[I 2024-11-08 11:26:34,198] Trial 2 finished with value: 454993000.0 and parameters: {'num_leaves': 28, 'learning_rate': 0.03703877249854557, 'min_data_in_leaf': 393, 'feature_fraction': 0.2450786254076267, 'bagging_fraction': 0.23471371488088055}. Best is trial 1 with value: 484022000.0.
[I 2024-11-08 11:28:04,561] Trial 3 finished with value: 489223000.0 and parameters: {'num_leaves': 79, 'learning_rate': 0.05486123634370735, 'min_data_in_leaf': 1380, 'feature_fraction': 0.618491354

KeyboardInterrupt: 

## Voting GD

In [114]:
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")

train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)

df_voting = pd.DataFrame()

for semilla in numeros_random:
    params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_params['num_leaves'],
    'learning_rate': study.best_params['learning_rate'],
    'min_data_in_leaf': study.best_params['min_data_in_leaf'],
    'feature_fraction': study.best_params['feature_fraction'],
    'bagging_fraction': study.best_params['bagging_fraction'],
    'seed': semilla,
    'verbose': 0
}
    
    
    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    df_voting[f'semilla_{semilla}'] = model.predict(X_test)

df_voting['promedio'] = df_voting.mean(axis=1)
    

Mejor cantidad de árboles para el mejor model 300


C:\Users\Admin\AppData\Local\Temp\ipykernel_2908\1798680220.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_voting['promedio'] = df_voting.mean(axis=1)


In [109]:
df_voting.head()

,semilla_400009,semilla_500009,semilla_500011,semilla_500021,semilla_600009,semilla_600011,semilla_600021,semilla_700009,semilla_7000011,semilla_700021,semilla_800009,semilla_800021,promedio
0,0.000305,0.000205,0.000324,0.000184,0.000170,0.000468,0.000516,0.000222,0.000336,0.000299,0.000455,0.000306,0.000316
1,0.000530,0.000835,0.000548,0.001046,0.000690,0.000491,0.001604,0.000435,0.000667,0.000317,0.001324,0.000446,0.000744
2,0.000171,0.000080,0.000144,0.000078,0.000072,0.000063,0.000088,0.000083,0.000087,0.000112,0.000114,0.000118,0.000101
3,0.083189,0.056778,0.058699,0.084973,0.089187,0.074947,0.102785,0.058986,0.059479,0.060757,0.101514,0.051234,0.073544
4,0.000491,0.000448,0.000976,0.000662,0.000644,0.000855,0.000754,0.000556,0.001379,0.000912,0.000992,0.000781,0.000788


In [115]:
df_voting.head()

,semilla_433,semilla_838,semilla_153,semilla_662,semilla_512,semilla_906,semilla_46,semilla_57,semilla_571,semilla_923,...,semilla_807,semilla_522,semilla_207,semilla_717,semilla_35,semilla_939,semilla_615,semilla_337,semilla_18,promedio
0,0.000084,0.000388,0.000331,0.000217,0.000987,0.000592,0.000241,0.000305,0.000517,0.002334,...,0.000446,0.000213,0.000221,0.000277,0.000079,0.000884,0.000189,0.000357,0.000151,0.000395
1,0.000217,0.000517,0.000281,0.001185,0.000171,0.000659,0.000571,0.001258,0.000692,0.000457,...,0.000330,0.000452,0.000244,0.000645,0.000721,0.000467,0.000337,0.000740,0.000483,0.000595
2,0.000036,0.000121,0.000256,0.000091,0.000047,0.000124,0.000047,0.000061,0.000072,0.000231,...,0.000168,0.000167,0.000061,0.000066,0.000053,0.000057,0.000075,0.000102,0.000055,0.000102
3,0.071394,0.109949,0.078699,0.068984,0.061558,0.078055,0.057936,0.074487,0.068497,0.072796,...,0.098121,0.062554,0.052918,0.083861,0.042690,0.084188,0.064770,0.046058,0.078592,0.072357
4,0.001316,0.000945,0.000550,0.001323,0.001013,0.000346,0.000951,0.001100,0.000790,0.001271,...,0.001490,0.000624,0.000841,0.000633,0.000928,0.000635,0.001135,0.001180,0.000885,0.000879


#### predicción

In [ ]:
y_pred_lgm = df_voting['promedio']

In [ ]:
y_pred_labels = (y_pred_lgm > 0.025).astype(int)


In [ ]:
#dataframe con las variables más importantes
importancias = model.feature_importance()
nombres = model.feature_name()
df_variables_importantes = pd.DataFrame({'importancia': importancias, 'variable': nombres})
df_variables_importantes = df_variables_importantes.sort_values(by='importancia', ascending=False)
lista_variables_importantes =  df_variables_importantes['variable'].head(10).to_list()

In [ ]:
df_resultado = pd.DataFrame({
    'numero_de_cliente': X_test['numero_de_cliente'],
    'Predicted': y_pred_labels
})

In [ ]:

# numero = '006'
# entrega = f'entrega_{numero}.csv'
# path = r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\Entregas"
# archivos = glob.glob(os.path.join(path, '*'))
# archivos.sort(key=os.path.getmtime, reverse=True)
# ultimo_archivo = archivos[0] if archivos else None
# if ultimo_archivo:
#     nombre_archivo = os.path.basename(ultimo_archivo)
#     numero = ''.join(filter(str.isdigit, nombre_archivo))
#     print(f"El último archivo entregado es: {nombre_archivo}")
#     print(f"El número extraído es: {numero}")
# else:
#     print("No se encontraron archivos en la carpeta.")


 

In [ ]:
#Ingresar el path sobre el cual se quiere ingresar el archivo a entregar

entrega = df_resultado.to_csv(r"C:\Users\Admin\Desktop\replicabilidad_competencia_01\competencia_01_replicabilidad.csv", index=False)

In [ ]:

# numero = '073'
# entrega = f'entrega_{numero}.csv'

In [ ]:


# if os.path.exists(path + entrega):
#     print("El archivo ya existe")
# else:
#     entrega_final = os.path.join(path, entrega)
#     df_resultado.to_csv(entrega_final, index=False)
    


In [ ]:
# entrega_final

In [ ]:
# cantidad_columnas = df_gbm.shape[1]
# message = f"{entrega}, cantidad de columnas en el train: {cantidad_columnas}, modelo: LGBM, mejores parametros: {best_params}, mejor iteracion: {best_iter}, archivo: {entrega_final}"

<!-- ### Kaggle -->

In [ ]:

# entrega_final = os.path.join(path, entrega)
# entrega_final
# competition= 'dm-ey-f-2024-primera'

In [ ]:
# kaggle.api.competition_submit(competition=competition, file_name= entrega_final, message=message, quiet=False)

### Puntos de corte

In [ ]:

# predicciones = y_pred_lgm

# X_test['Probabilidad'] = predicciones

# tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)

# cortes = range(9000,14000,100)

# num_subida_kaggle = 65
# for envios in cortes:
    
#     tb_entrega['Predicted'] = 0
#     tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
#     resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
    
#     print("Cantidad de clientes {}".format(envios))
    
#     nombre_archivo = 'entrega_0{}.csv'.format(num_subida_kaggle)
#     entrega_final = os.path.join(path, nombre_archivo)
#     resultados.to_csv(entrega_final, index=False)
    
    
#     cantidad_columnas = df_train.shape[1]
#     message = f"{entrega}, cantidad de columnas en el train: {cantidad_columnas}, modelo: LGBM, mejores parametros: {best_params}, mejor iteracion: {best_iter}, archivo: {entrega_final}, punto de corte: {envios}, optimizado con optuna: {study_name}"
    
#     num_subida_kaggle += 1
    
    
    
#     entrega_final = os.path.join(path, nombre_archivo)   
#     competencia = 'dm-ey-f-2024-primera'
#     try:
#         api.competition_submit(file_name=entrega_final,message=message,competition=competencia)
#     except:
#         print(f"Numero máximo de envios, último envio ={num_subida_kaggle}")

In [ ]:
# df_train.shape

In [ ]:
# nombre_modelo = 'lgbm_e_en_abril_p_en_junio_451_features.txt'
# model.save_model(rf"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\modelos_lgbm\{nombre_modelo}")